In [55]:
import normals_lib
import pointareas
import kaolin as kal
import torch
import numpy as np
import meshplot as mp

#mesh = kal.rep.TriangleMesh.from_obj("model.obj") # Modelo de una esfera
#mesh = kal.rep.TriangleMesh.from_obj("cow.obj") # Modelo de vaca
#mesh = kal.rep.TriangleMesh.from_obj("torus.obj")
mesh = kal.rep.TriangleMesh.from_obj("simplified_bunny.obj")

In [56]:
view_coords = torch.load("view_coordinates.pt").to(device=torch.device('cuda'))
verts = mesh.vertices.clone()
xmed = torch.median(verts[:,0])
ymed = torch.median(verts[:,1])
zmed = torch.median(verts[:,2])
verts[:,0] -= xmed
verts[:,1] -= ymed
verts[:,2] -= zmed
verts /= torch.max(torch.norm(verts, dim=1))
mesh.vertices = verts

In [57]:
normals = normals_lib.compute_simple_vertex_normals(mesh).to(device=torch.device('cpu'))
pointareas, cornerareas = pointareas.compute_pointareas(mesh)
normals_lib.visualize_normals(mesh, normals)
normals = normals.to(device=torch.device('cuda'))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1351746…

In [58]:
# Calculate principal curvatures and curvature directions
import curvature

k1, k2, e1, e2 = curvature.compute_curvatures(mesh, normals=normals, pointareas=pointareas,
                                              cornerareas=cornerareas)

In [59]:
import dcurv

dc = dcurv.compute_dcurvs(mesh, curvs=(k1,k2,e1,e2), normals=normals, pointareas=pointareas,
                          cornerareas=cornerareas)

In [81]:
import importlib
importlib.reload(dcurv)

<module 'dcurv' from '/home/danniccs/rtsc_to_torch/dcurv.py'>

In [113]:
DwKr = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
kr = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
for i in range(0, 12):
    DwKr[i], kr[i] = dcurv.compute_DwKr(mesh, normals=normals, curvs=(k1,k2,e1,e2), dcurv=dc,
                                        view_coords=view_coords[i])

In [47]:
tridc = torch.zeros(dc.shape)
with open("/home/danniccs/rtsc-1.6/dcurv", "r") as myfile:
    for i in range(0, dc.shape[0]):
        l = myfile.readline()
        tridc[i] = torch.tensor([float(n) for n in l[1:-2].split(',')])
DwKrtri, krtri = dcurv.compute_DwKr(mesh, normals=normals, curvs=(k1,k2,e1,e2), dcurv=tridc.to(device=torch.device('cuda')))

In [83]:
print(DwKr[0].min())
print(DwKr[0].mean())
print(DwKr[0].max())
print(kr[0].mean())
print(kr[0].min())
print(kr[0].max())

tensor(-27759.0547, device='cuda:0')
tensor(20.0489, device='cuda:0')
tensor(23948.0566, device='cuda:0')
tensor(2.9970, device='cuda:0')
tensor(-54.9730, device='cuda:0')
tensor(70.8708, device='cuda:0')


In [7]:
dwkr_copy = DwKr.clone()

In [55]:
#DwKr = dwkr_copy
(DwKr[0][DwKr[0] > DwKr[0].mean()]).sum()
DwKr[0]
DwKr[0].mean()

tensor(nan, device='cuda:0')

In [40]:
import matplotlib
matplotlib.use('TkAgg')
plt = matplotlib.pyplot
import math
DwKr[0][DwKr[0] >= DwKr[0].mean() * 2] = DwKr[0].mean() * 1.5
DwKr[0][DwKr[0].mean() * 0.5 >= DwKr[0]] = DwKr[0].mean() * 0.5
DwKr[0][torch.isinf(DwKr[0])] = 0.0
n, bins, patches = plt.hist(DwKr[0].cpu())
plt.show()

In [30]:
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=(k2 * k1).to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1351746…

In [87]:
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=k1.to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1351746…

In [122]:
import shared_algs
importlib.reload(shared_algs)
ndotv, krf, viewdir, sctest_num, _ = shared_algs.compute_perview(mesh, normals=normals, curvs=(k1,k2,e1,e2),
                                                                 dcurv=dc, view_coords=view_coords[3])
w = viewdir - normals * ndotv[:,None]

In [123]:
scupper = sctest_num.mean() + abs(sctest_num.mean() * 3)
sclower = sctest_num.mean() - abs(sctest_num.mean() * 3)
kupper = int(sctest_num.shape[0] * 0.9)
klower = int(sctest_num.shape[0] * 0.1)
scupper, _ = torch.kthvalue(sctest_num, kupper)
sclower, _ = torch.kthvalue(sctest_num, klower)
sctest_num[sctest_num > scupper] = scupper
sctest_num[sctest_num < sclower] = sclower
print(sctest_num.min())
print(sctest_num.max())
print(sctest_num.mean())
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=sctest_num.to(device=torch.device('cpu')).numpy(),
        shading={'colormap': "jet_r"})

tensor(-143.9539, device='cuda:0')
tensor(154.6957, device='cuda:0')
tensor(1.7263, device='cuda:0')


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1351746…

In [114]:
for i in range(3, 4):
    kupper = int(DwKr[i].shape[0] * 0.9)
    klower = int(DwKr[i].shape[0] * 0.1)
    dwupper, _ = torch.kthvalue(DwKr[i], kupper)
    dwlower, _ = torch.kthvalue(DwKr[i], klower)
    DwKr[i][DwKr[i] > dwupper] = dwupper
    DwKr[i][DwKr[i] < dwlower] = dwlower
    mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr[i].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1351746…

In [115]:
for i in range(3, 4):
    kupper = int(kr[i].shape[0] * 0.9)
    klower = int(kr[i].shape[0] * 0.1)
    krupper, _ = torch.kthvalue(kr[i], kupper)
    krlower, _ = torch.kthvalue(kr[i], klower)
    kr[i][kr[i] > krupper] = krupper
    mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=kr[i].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1351746…

In [33]:
kr[2].max()

tensor(4.8217, device='cuda:0')

In [11]:
view_coords

tensor([[ 2.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 1.7321e+00,  0.0000e+00,  1.0000e+00],
        [ 1.0000e+00,  0.0000e+00,  1.7321e+00],
        [ 1.2246e-16,  0.0000e+00,  2.0000e+00],
        [-1.0000e+00,  0.0000e+00,  1.7321e+00],
        [-1.7321e+00,  0.0000e+00,  1.0000e+00],
        [-2.0000e+00,  0.0000e+00,  2.4493e-16],
        [-1.7321e+00,  0.0000e+00, -1.0000e+00],
        [-1.0000e+00,  0.0000e+00, -1.7321e+00],
        [-3.6739e-16,  0.0000e+00, -2.0000e+00],
        [ 1.0000e+00,  0.0000e+00, -1.7321e+00],
        [ 1.7321e+00,  0.0000e+00, -1.0000e+00]], device='cuda:0')

In [9]:
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=mesh.vertices[:,1].to(device=torch.device('cpu')).numpy(),
        shading={'colormap': "jet"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1172533…